In [17]:
import pandas as pd
from datetime import timedelta
from datetime import time


from kiblib.utils.db import DbConn

In [29]:
db_conn = DbConn().create_engine()
date = "2023-01-01"

In [36]:
query = f"""SELECT
    o.issue_id,
    o.issuedate,
    o.branchcode as branch,
    o.borrowernumber,
    b.cardnumber,
    CASE
        WHEN b.categorycode NOT IN ('BIBL', 'CSLT','CSVT','MEDA','MEDB','MEDC','MEDP')
            THEN 'NP'
            ELSE YEAR(o.issuedate) - YEAR(b.dateofbirth)
    END as age,
    CASE
        WHEN b.title = 'Madame'
            THEN 'F'
        WHEN b.title = 'Monsieur'
            THEN 'M'
        WHEN b.categorycode NOT IN ('BIBL','CSLT','CSVT','MEDA','MEDB','MEDC','MEDP')
            THEN 'NP'
    END as sexe,
    b.city as ville,
    b.altcontactcountry as iris,
    b.branchcode,
    b.categorycode,
    YEAR(o.issuedate) - YEAR(b.dateenrolled) as fidelite,
    o.itemnumber,
    i.homebranch,
    i.location,
    i.ccode,
    i.itemcallnumber,
    bi.itemtype,
    bi.publicationyear,
    i.biblionumber,
    i.dateaccessioned,
    o.issue_id
FROM koha_prod.issues o
LEFT JOIN koha_prod.borrowers b ON o.borrowernumber = b.borrowernumber
LEFT JOIN koha_prod.items i ON o.itemnumber = i.itemnumber
LEFT JOIN koha_prod.biblioitems bi ON i.biblionumber = bi.biblionumber
WHERE DATE(o.issuedate) >= '{date}'"""

issues1 = pd.read_sql(query, db_conn)

query = f"""SELECT
    o.issue_id,
    o.issuedate,
    o.branchcode as branch,
    o.borrowernumber,
    b.cardnumber,
    CASE
        WHEN b.categorycode NOT IN ('BIBL', 'CSLT','CSVT','MEDA','MEDB','MEDC','MEDP')
            THEN 'NP'
            ELSE YEAR(o.issuedate) - YEAR(b.dateofbirth)
    END as age,
    CASE
        WHEN b.title = 'Madame'
            THEN 'F'
        WHEN b.title = 'Monsieur'
            THEN 'M'
        WHEN b.categorycode NOT IN ('BIBL','CSLT','CSVT','MEDA','MEDB','MEDC','MEDP')
            THEN 'NP'
    END as sexe,
    b.city as ville,
    b.altcontactcountry as iris,
    b.branchcode,
    b.categorycode,
    YEAR(o.issuedate) - YEAR(b.dateenrolled) as fidelite,
    o.itemnumber,
    i.homebranch,
    i.location,
    i.ccode,
    i.itemcallnumber,
    bi.itemtype,
    bi.publicationyear,
    i.biblionumber,
    i.dateaccessioned,
    o.issue_id
FROM koha_prod.old_issues o
LEFT JOIN koha_prod.borrowers b ON o.borrowernumber = b.borrowernumber
LEFT JOIN koha_prod.items i ON o.itemnumber = i.itemnumber
LEFT JOIN koha_prod.biblioitems bi ON i.biblionumber = bi.biblionumber
WHERE DATE(o.issuedate) >= '{date}'"""

issues2 = pd.read_sql(query, db_conn)

issues = pd.concat([issues1, issues2])
print(len(issues))

issues['arret_bus'] = 'NP'
issues.loc[issues['branch'] == 'BUS', 'arret_bus'] = 'NP'
issues['issuedate'] = pd.to_datetime(issues['issuedate'])

248947


In [37]:
issues_med = issues[issues['branch'] != 'BUS']
issues_bus = issues[issues['branch'] == 'BUS']

In [39]:
weekdays = {
           'lundi':0,
           'mardi':1,
           'mercredi':2,
           'jeudi':3,
           'vendredi':4,
           'samedi':5,
           'dimanche':6
           }

# mardis 14h00 - 15h00 / Rue Louis Braille, quartier Linne-Chemin neuf / B01
issues_bus.loc[(issues['issuedate'].dt.weekday == weekdays['mardi'])
               & (issues['issuedate'].dt.time >= time(hour=13, minute=50))
               & (issues['issuedate'].dt.time <= time(hour=14, minute=8)), 'arret_bus'] = 'B01'

# mardis 15h15 - 16h15 / Place de la Fraternité, quartier Fraternité / B13
issues_bus.loc[(issues['issuedate'].dt.weekday == weekdays['mardi'])
               & (issues['issuedate'].dt.time >= time(hour=14, minute=9))
               & (issues['issuedate'].dt.time <= time(hour=16, minute=23)), 'arret_bus'] = 'B13'

# mardis 16h30 - 18h00 / Place du Travail, quartier Moulin Potennerie / B03
issues_bus.loc[(issues['issuedate'].dt.weekday == weekdays['mardi'])
               & (issues['issuedate'].dt.time >= time(hour=16, minute=24))
               & (issues['issuedate'].dt.time <= time(hour=18, minute=8)), 'arret_bus'] = 'B03'

# mercredis 9h30 - 10h30 / Rue Drouot, quartier de l'Hommelet / B24
issues_bus.loc[(issues['issuedate'].dt.weekday == weekdays['mercredi'])
               & (issues['issuedate'].dt.time >= time(hour=9, minute=24))
               & (issues['issuedate'].dt.time <= time(hour=10, minute=38)), 'arret_bus'] = 'B24'

# mercredis 10h45 - 11h45 / Rue Jacques Prévert, quartier Sainte Elisabeth / B14
issues_bus.loc[(issues['issuedate'].dt.weekday == weekdays['mercredi'])
               & (issues['issuedate'].dt.time >= time(hour=9, minute=39))
               & (issues['issuedate'].dt.time <= time(hour=11, minute=55)), 'arret_bus'] = 'B14'

# mercredis 14h00 - 15h00 / Place Carnot, quartier du Pile / B07
issues_bus.loc[(issues['issuedate'].dt.weekday == weekdays['mercredi'])
               & (issues['issuedate'].dt.time >= time(hour=13, minute=53))
               & (issues['issuedate'].dt.time <= time(hour=15, minute=8)), 'arret_bus'] = 'B07'

# mercredis 15h15 - 16h15 / Rue de France, quartier de l'Alma / B08
issues_bus.loc[(issues['issuedate'].dt.weekday == weekdays['mercredi'])
               & (issues['issuedate'].dt.time >= time(hour=15, minute=8))
               & (issues['issuedate'].dt.time <= time(hour=16, minute=23)), 'arret_bus'] = 'B08'

# mercredis 16h30 - 17h30 / Rue de Rome, quartier du Fresnoy Mackellerie / B09
issues_bus.loc[(issues['issuedate'].dt.weekday == weekdays['mercredi'])
               & (issues['issuedate'].dt.time >= time(hour=16, minute=24))
               & (issues['issuedate'].dt.time <= time(hour=17, minute=38)), 'arret_bus'] = 'B09'

# vendredis 16h00 - 17h00 / Rue de Philippeville, quartier Hutin-Oran-Cartigny / B23
issues_bus.loc[(issues['issuedate'].dt.weekday == weekdays['vendredi'])
               & (issues['issuedate'].dt.time >= time(hour=15, minute=53))
               & (issues['issuedate'].dt.time <= time(hour=17, minute=10)), 'arret_bus'] = 'B23'

# samedi 10h30 - 11h30/ Place de la Citoyenneté, quartier des Trois Ponts / B26
issues_bus.loc[(issues['issuedate'].dt.weekday == weekdays['samedi'])
               & (issues['issuedate'].dt.time >= time(hour=10, minute=23))
               & (issues['issuedate'].dt.time <= time(hour=11, minute=40)), 'arret_bus'] = 'B26'

# samedi 14h00 - 15h00 / Rue Montgolfier, quartier Moulin-Potennerie / B17
issues_bus.loc[(issues['issuedate'].dt.weekday == weekdays['samedi'])
               & (issues['issuedate'].dt.time >= time(hour=13, minute=53))
               & (issues['issuedate'].dt.time <= time(hour=15, minute=8)), 'arret_bus'] = 'B17'

# samedi 15h15 - 16h30 / Boulevard de Fourmies, quartier du Nouveau Roubaix B19
issues_bus.loc[(issues['issuedate'].dt.weekday == weekdays['samedi'])
               & (issues['issuedate'].dt.time >= time(hour=15, minute=9))
               & (issues['issuedate'].dt.time <= time(hour=16, minute=40)), 'arret_bus'] = 'B19'



/tmp/ipykernel_3365/312713294.py:12: FutureWarning: reindexing with a non-unique Index is deprecated and will raise in a future version.
  issues_bus.loc[(issues['issuedate'].dt.weekday == weekdays['mardi'])


ValueError: cannot reindex on an axis with duplicate labels

In [40]:
issues_bus['arret_bus'].value_counts()

NP    10246
Name: arret_bus, dtype: int64

In [34]:
issues_med.to_sql('stat_issues_test', db_conn, if_exists='append', index=False)
issues_bus.to_sql('stat_issues_test', db_conn, if_exists='append', index=False)

616

In [41]:
query = f"""SELECT
    o.issue_id,
    o.returndate,
    o.date_due,
    o.renewals
FROM koha_prod.old_issues o
WHERE DATE(o.returndate) >= '{date}'
AND DATE(o.issuedate) >= '{date}'"""

returns = pd.read_sql(query, db_conn)

In [42]:
returns

,issue_id,returndate,date_due,renewals
0,3086842,2023-02-07 12:29:06,2023-02-14 23:59:00,1
1,3086843,2023-01-14 10:00:13,2023-01-24 23:59:00,0
2,3086844,2023-01-14 10:00:05,2023-01-24 23:59:00,0
3,3086845,2023-01-10 09:22:28,2023-01-24 23:59:00,0
4,3086846,2023-02-02 09:17:09,2023-02-14 23:59:00,1
...,...,...,...,...
218151,3335237,2023-09-12 14:19:23,2023-10-03 23:59:00,0
218152,3335238,2023-09-12 14:19:14,2023-10-03 23:59:00,0
218153,3335239,2023-09-12 14:19:21,2023-10-03 23:59:00,0
218154,3335619,2023-09-12 17:44:22,2023-10-03 23:59:00,0
